<a href="https://colab.research.google.com/github/AMDB900/234Project/blob/main/234_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pycryptodome # Install the correct package using pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 16.2 MB/s eta 0:00:00


In [3]:
import os
import random
import string
import hashlib
from Crypto.Cipher import AES

# ---------------- STEP 1: Create a Sample Text File ----------------
def create_sample_text_file(filename, text):
    """Creates a text file with given text."""
    with open(filename, 'w') as file:
        file.write(text)
    print(f"✅ Sample text file '{filename}' created!")

# ---------------- STEP 2: Generate Random Encryption Key & IV ----------------
def generate_random_key(length=32):
    """Generates a secure random encryption key."""
    return ''.join(random.choices(string.ascii_letters + string.digits + "QWERTYUIOPASDFGHJKLZXCVBNM1234567890!@#$%^&*", k=length))

def generate_random_iv():
    """Generates a 16-byte random IV for AES encryption."""
    return os.urandom(16)

# ---------------- STEP 3: Encrypt the File ----------------
def encrypt_file(input_filename):
    """Encrypts a file using AES-256 CBC encryption."""
    if not os.path.exists(input_filename):
        print(f"❌ Error: File '{input_filename}' not found!")
        return None

    with open(input_filename, 'rb') as file:
        data = file.read()

    # Step 1: Generate Random Key & IV
    encryption_key = generate_random_key()
    iv = generate_random_iv()

    # Step 2: Convert key into a 256-bit hash
    key_hash = hashlib.sha256(encryption_key.encode()).digest()

    # Step 3: Ensure data is padded to a multiple of 16 bytes
    padding_length = 16 - (len(data) % 16)
    padded_data = data + bytes([padding_length]) * padding_length

    # Step 4: Encrypt using AES-256 CBC mode
    cipher = AES.new(key_hash, AES.MODE_CBC, iv)
    ciphertext = cipher.encrypt(padded_data)

    # Step 5: Save encrypted file (IV + ciphertext)
    encrypted_filename = input_filename + ".enc"
    with open(encrypted_filename, 'wb') as file:
        file.write(iv + ciphertext)

    print(f"✅ File '{input_filename}' encrypted and saved as '{encrypted_filename}'!")
    return encrypted_filename, encryption_key  # Return filename & key for decryption

# ---------------- STEP 4: Decrypt the File ----------------
def decrypt_file(encrypted_filename, encryption_key):
    """Decrypts an AES-256 CBC encrypted file and restores original content."""
    if not os.path.exists(encrypted_filename):
        print(f"❌ Error: File '{encrypted_filename}' not found!")
        return None

    # Step 1: Convert key into 256-bit hash
    key_hash = hashlib.sha256(encryption_key.encode()).digest()

    with open(encrypted_filename, 'rb') as file:
        iv = file.read(16)  # Extract IV
        ciphertext = file.read()

    # Step 2: Decrypt the file
    cipher = AES.new(key_hash, AES.MODE_CBC, iv)
    decrypted_padded_data = cipher.decrypt(ciphertext)

    # Step 3: Remove padding
    padding_length = decrypted_padded_data[-1]
    decrypted_data = decrypted_padded_data[:-padding_length]

    # Step 4: Save decrypted file
    decrypted_filename = encrypted_filename.replace(".enc", "_decrypted.txt")
    with open(decrypted_filename, 'wb') as file:
        file.write(decrypted_data)

    print(f"✅ File '{encrypted_filename}' decrypted and saved as '{decrypted_filename}'!")
    return decrypted_filename

# ---------------- RUN ENCRYPTION & DECRYPTION ----------------
# Step 1: Create a sample text file
sample_text = "This is a simple encryption test. Let's see if it works correctly!"
text_filename = "test.txt"
create_sample_text_file(text_filename, sample_text)

# Step 2: Encrypt the file
encrypted_file, key = encrypt_file(text_filename)

# Step 3: Decrypt the file using the same key
if encrypted_file:
    decrypted_file = decrypt_file(encrypted_file, key)

    # Step 4: Verify if decryption worked
    with open(decrypted_file, 'r') as file:
        decrypted_content = file.read()
    print("\n🔍 Decrypted Content:")
    print(decrypted_content)


✅ Sample text file 'test.txt' created!
✅ File 'test.txt' encrypted and saved as 'test.txt.enc'!
✅ File 'test.txt.enc' decrypted and saved as 'test.txt_decrypted.txt'!

🔍 Decrypted Content:
This is a simple encryption test. Let's see if it works correctly!


In [4]:
import heapq
import os
import random
import string
from collections import Counter, namedtuple

# ---------------- STEP 1: Create a Sample Text File ----------------

def create_sample_text_file(filename, text):
    """Creates a text file with given text."""
    with open(filename, 'w') as file:
        file.write(text)
    print(f"✅ Sample text file '{filename}' created!")

# ---------------- STEP 2: Huffman Encoding Functions ----------------

class Node(namedtuple("Node", ["char", "freq", "left", "right"])):
    def __lt__(self, other):
        return self.freq < other.freq

def build_huffman_tree(text):
    """Builds the Huffman tree for encoding."""
    freq = Counter(text)
    heap = [Node(char, freq, None, None) for char, freq in freq.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        merged = Node(None, left.freq + right.freq, left, right)
        heapq.heappush(heap, merged)

    return heap[0]

def generate_huffman_codes(node, prefix="", codebook={}):
    """Generates Huffman encoding dictionary."""
    if node:
        if node.char:
            codebook[node.char] = prefix
        generate_huffman_codes(node.left, prefix + "0", codebook)
        generate_huffman_codes(node.right, prefix + "1", codebook)
    return codebook

# ---------------- STEP 3: Huffman Encrypt (Compress) ----------------

def encrypt_file(input_filename):
    """Encodes a file using Huffman compression."""
    if not os.path.exists(input_filename):
        print(f"❌ Error: File '{input_filename}' not found!")
        return None

    with open(input_filename, 'r') as file:
        text = file.read()

    # Step 1: Build Huffman Tree & Generate Codes
    huffman_tree = build_huffman_tree(text)
    codes = generate_huffman_codes(huffman_tree)

    # Step 2: Encode text using Huffman codes
    encoded_text = ''.join(codes[char] for char in text)

    # Step 3: Save compressed file
    encrypted_filename = input_filename + ".huff"
    with open(encrypted_filename, 'w') as file:
        file.write(encoded_text)

    print(f"✅ File '{input_filename}' compressed and saved as '{encrypted_filename}'!")
    return encrypted_filename, huffman_tree  # Return file & Huffman tree for decoding

# ---------------- STEP 4: Huffman Decrypt (Decompress) ----------------

def decrypt_file(encrypted_filename, huffman_tree):
    """Decodes a Huffman compressed file back to original."""
    if not os.path.exists(encrypted_filename):
        print(f"❌ Error: File '{encrypted_filename}' not found!")
        return None

    with open(encrypted_filename, 'r') as file:
        encoded_text = file.read()

    # Decode Huffman encoding
    decoded_text = []
    node = huffman_tree
    for bit in encoded_text:
        node = node.left if bit == "0" else node.right
        if node.char:
            decoded_text.append(node.char)
            node = huffman_tree

    # Save the decompressed file
    decrypted_filename = encrypted_filename.replace(".huff", "_decompressed.txt")
    with open(decrypted_filename, 'w') as file:
        file.write(''.join(decoded_text))

    print(f"✅ File '{encrypted_filename}' decompressed and saved as '{decrypted_filename}'!")
    return decrypted_filename

# ---------------- RUN HUFFMAN ENCRYPTION & DECRYPTION ----------------

# Step 1: Create a sample text file
sample_text = "This is a simple encryption test. Let's see if Huffman compression works correctly!"
text_filename = "test.txt"
create_sample_text_file(text_filename, sample_text)

# Step 2: Encrypt (Compress) the file using Huffman encoding
encrypted_file, tree = encrypt_file(text_filename)

# Step 3: Decrypt (Decompress) the file using Huffman decoding
if encrypted_file:
    decrypted_file = decrypt_file(encrypted_file, tree)

    # Step 4: Verify if decompression worked
    with open(decrypted_file, 'r') as file:
        decrypted_content = file.read()
    print("\n🔍 Decrypted Content:")
    print(decrypted_content)


✅ Sample text file 'test.txt' created!
✅ File 'test.txt' compressed and saved as 'test.txt.huff'!
✅ File 'test.txt.huff' decompressed and saved as 'test.txt_decompressed.txt'!

🔍 Decrypted Content:
This is a simple encryption test. Let's see if Huffman compression works correctly!
